# CONV NETS Testing

In [30]:
import numpy as np
import h5py
from utils_cnn.utils import load_dataset
from utils_cnn.utils import convert_to_one_hot
from utils_cnn.forward import conv_layer
from utils_cnn.forward import pool_forward

In [24]:
(X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes) = load_dataset()

In [26]:
X_train = X_train_orig/255.
X_test = X_test_orig/255.
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))

number of training examples = 1080
number of test examples = 120


## Convolutional Layer

In [27]:
np.random.seed(1)
A_prev = np.random.randn(10,4,4,3)
W = np.random.randn(3,3,3,8)
b = np.random.randn(1,1,1,8)
parameters = {"pad" : 1,
               "stride": 1}

Z, cache_conv = conv_layer(X_train[:50], W, b, {"pad": 1, "stride": 1})
print("Z's mean =", np.mean(Z))

("Z's mean =", 0.91593632418013071)


## MAX Pooling Layer

In [34]:
A, cache_pool = pool_forward(Z, {"stride" : 2, "f": 2}, mode="max")

In [36]:
A.shape

(50, 32, 32, 8)